In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('/kaggle/input/dpl-2025/Text_Image.csv')
df_test = pd.read_csv('/kaggle/input/dpl-2025/Test_Text_Image.csv')

base_image_dir_train = "/kaggle/input/dpl-2025/devset_images" 
base_image_dir_test = "/kaggle/input/dpl-2025/testset_images" 

df_train['image_path'] = df_train['image_path'].apply(lambda x: f"{base_image_dir_train}/{x}")
df_test['image_path'] = df_test['image_path'].apply(lambda x: f"{base_image_dir_test}/{x}")



In [3]:
df_train = df_train.fillna("missing") 
def fill_missing(text):
    if isinstance(text, str) and text.strip() == "":
        return "missing"
    return text
df_train = df_train.applymap(fill_missing)
df_train.head()

df_test = df_test.fillna("missing")
df_test = df_test.applymap(fill_missing)
df_test.head()

/tmp/ipykernel_35/3514959801.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_train = df_train.applymap(fill_missing)
/tmp/ipykernel_35/3514959801.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_test = df_test.applymap(fill_missing)


,image_id,title,description,user_tags,image_path
0,3483809003,"Flooded Parking Lot At Emily Fowler Library, A...",Denton Creek overflows its banks and floods Oa...,"project, slis 5715, spring 2009",/kaggle/input/dpl-2025/testset_images/testset_...
1,3712805295,L'arc de Barà / The roman arch of Barà,Sembla que fou dedicat a August entorn l'any 1...,"arc, arc_de_berà, arch, archaeology, arco, arq...",/kaggle/input/dpl-2025/testset_images/testset_...
2,379845620,Highest point over the sea level that is reach...,missing,missing,/kaggle/input/dpl-2025/testset_images/testset_...
3,7343264988,Lagos after the rains,"After heavy rain, Lagos (Nigeria) was still fl...","africa, lagos, nigeria",/kaggle/input/dpl-2025/testset_images/testset_...
4,3843337492,flooded Corley Ave,also a local black out due to the tree branch ...,"flood, storm, toronto",/kaggle/input/dpl-2025/testset_images/testset_...


In [4]:
texts = (
    df_train['title'].fillna('') + ' ' +
    df_train['description'].fillna('') + ' ' +
    df_train['user_tags'].fillna('')
).tolist()
image_paths_train = df_train['image_path'].tolist()
labels = df_train['label'].tolist()  # Assuming labels are in a column named 'label'

test_ids = df_test['image_id'].tolist()
test_texts = (
    df_test['title'].fillna('') + ' ' +
    df_test['description'].fillna('') + ' ' +
    df_test['user_tags'].fillna('')
).tolist()
image_paths_test = df_test['image_path'].tolist()

train_texts, val_texts, train_labels, val_labels, train_image_paths, val_image_paths = train_test_split(texts, labels, image_paths_train, 
                                                                                      test_size=0.05, stratify=labels, random_state=42)

# Bert + ViT

In [5]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)

        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Save model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} → {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [6]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from tqdm import tqdm
import torch

from PIL import Image, ImageFile
import os
ImageFile.LOAD_TRUNCATED_IMAGES = True

class MultiModalDataset(Dataset):
    def __init__(self, texts, labels=None, image_paths=None, tokenizer=None, image_transform=None, max_length=128):
        self.tokenizer = tokenizer
        self.image_transform = image_transform
        self.max_length = max_length
        self.texts = texts
        self.labels = labels
        self.image_paths = image_paths
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        image_path = self.image_paths[idx]

        image = Image.open(image_path).convert("RGB")
        image = self.image_transform(image)

        encoding = self.tokenizer(text, max_length=self.max_length,
                                  padding='max_length', truncation=True, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze(0)  
        attention_mask = encoding['attention_mask'].squeeze(0)

        item = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'pixel_values': image
        }

        if self.labels is not None:
            label = torch.tensor(self.labels[idx])
            item['labels'] = label

        return item

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

image_transform =transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.RandomHorizontalFlip(), #Data argumentation
    transforms.ColorJitter(), #Data argumentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
train_dataset = MultiModalDataset(train_texts, train_labels, train_image_paths, tokenizer, image_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

val_dataset = MultiModalDataset(val_texts, val_labels, val_image_paths, tokenizer, image_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)

test_dataset = MultiModalDataset(test_texts, None, image_paths_test, tokenizer, image_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [8]:
import torch
import torch.nn as nn
from transformers import BertModel, ViTModel
from transformers.modeling_outputs import SequenceClassifierOutput

class CombinedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224')

        bert_hidden = self.bert.config.hidden_size
        vit_hidden = self.vit.config.hidden_size

        self.classifier = nn.Sequential(
            nn.Linear(bert_hidden + vit_hidden, 512), #Tổng chiều đầu ra của torch.cat((bert_cls, vit_cls), dim=1) = bert_dim + vit_dim → Phải khớp với input size của nn.Linear
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 2)
        )
    
    def forward(self, input_ids, attention_mask, pixel_values):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_cls = bert_out.last_hidden_state[:, 0, :]  # CLS token

        vit_out = self.vit(pixel_values=pixel_values)
        vit_cls = vit_out.last_hidden_state[:, 0, :]

        combined = torch.cat((bert_cls, vit_cls), dim=1)
        logits = self.classifier(combined)
        #return logits.squeeze(1) # batch size = 1: return logits.view(-1)
        return SequenceClassifierOutput(logits=logits)


2025-06-02 07:37:30.080539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748849850.475533      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748849850.593684      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
def validation(model, dataloader, criterion, device):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0

    with torch.no_grad():
        loop = tqdm(val_loader, leave=True)
        for batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, pixel_values)
            logits = outputs.logits
            loss = criterion(logits, labels)

            #preds = torch.sigmoid(outputs) >= 0.5       
            preds = torch.argmax(logits, dim=1)
                        
            total_loss += loss.item() * labels.size(0)
            total_correct += (preds == labels.long()).sum().item()
            total_samples += labels.size(0)
 
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CombinedModel().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
criterion = nn.CrossEntropyLoss()

num_epochs = 10
model.train()
early_stopper = EarlyStopping(patience=3, verbose=True)

for epoch in range(num_epochs):
    loop = tqdm(train_loader, leave=True)
    total_loss, total_accuracy, total_samples = 0, 0, 0
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, pixel_values)
        logits = outputs.logits  # Remove it if not use huggingface
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()


        # probs = torch.sigmoid(outputs)
        # preds = (probs > 0.5).long()
        preds = torch.argmax(logits, dim=1)
        
        total_loss += loss.item() * labels.size(0)
        total_accuracy += (preds == labels.long()).sum().item()
        total_samples += labels.size(0)
        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())  
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss/total_samples:.4f}, Train Accuracy: {total_accuracy/total_samples:.4f}")

    val_loss, val_acc = validation(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}: Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

    early_stopper(val_loss, model)
    if early_stopper.early_stop:
        print("Early stopping")
        break


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 157/157 [04:48<00:00,  1.84s/it, loss=0.239]


Epoch [1/10], Train Loss: 0.5209, Train Accuracy: 0.7438


...

Epoch 5: 100%|██████████| 157/157 [04:53<00:00,  1.87s/it, loss=0.024]  


Epoch [5/10], Train Loss: 0.0129, Train Accuracy: 0.9986


Epoch [5/10]: 100%|██████████| 9/9 [00:06<00:00,  1.47it/s]


Epoch 5: Val Loss=0.1596, Val Acc=0.9432
EarlyStopping counter: 2 out of 3


Epoch 6:  80%|████████  | 126/157 [03:55<00:57,  1.86s/it, loss=0.00145] 

In [ ]:
model.eval()
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to('cuda' if torch.cuda.is_available() else 'cpu')
        attention_mask = batch['attention_mask'].to('cuda' if torch.cuda.is_available() else 'cpu')
        pixel_values = batch['pixel_values'].to('cuda' if torch.cuda.is_available() else 'cpu')

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        
        # probs = torch.sigmoid(outputs).squeeze().cpu().numpy()
        # preds = (probs > 0.5).astype(int)
        logits = outputs.logits # if use model huggingface to return result need .logit to get output
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())

df_test['label'] = all_preds
df_test[['image_id', 'label']].to_csv("predictions.csv", index=False)
